In [2]:
!apt-get install clisp
!apt-get install gcc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libffcall1b
Suggested packages:
  gdb clisp-doc slime clisp-module-berkeley-db clisp-module-clx
  clisp-module-dbus clisp-module-fastcgi clisp-module-gdbm clisp-module-libsvm
  clisp-module-pari clisp-module-pcre clisp-module-postgresql
  clisp-module-zlib hyperspec
The following NEW packages will be installed:
  clisp libffcall1b
0 upgraded, 2 newly installed, 0 to remove and 41 not upgraded.
Need to get 6,424 kB of archives.
After this operation, 33.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libffcall1b amd64 2.4-1 [12.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 clisp amd64 1:2.49.20210628.gitde01f0f-2 [6,411 kB]
Fetched 6,424 kB in 2s (4,032 kB/s)
Selecting previously unselected package libffcall1b:amd64.
(Reading database ... 121713 files and director

In [3]:
import subprocess

# load and split Lisp programs
with open("dataset.lisp", "r", encoding="utf-8") as f:
    data = f.read()

programs = [p.strip() for p in data.split("##") if p.strip()]

print("Loaded", len(programs), "program(s).")


# compile and run each program using clisp and store the outputs
results = []

for i, program in enumerate(programs):
    process = subprocess.run(
        ["clisp", "-q"],
        input=program.encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    output = process.stdout.decode("utf-8").strip()
    error  = process.stderr.decode("utf-8").strip()

    results.append(output)

# print outputs
print("\nResults:")
for i, result in enumerate(results):
    print(f"\nProgram {i+1} output:")
    print(result)


Loaded 25 program(s).

Results:

Program 1 output:
[1]> 
COMPRESS
[2]> 
COMPR
[3]> 
N-ELTS
[4]> 
((3 1) 0 1 (4 0) 1) 
((3 1) 0 1 (4 0) 1)

Program 2 output:
[1]> 
UNCOMPRESS
[2]> 
LIST-OF
[3]> 
(1 1 1 0 1 0 0 0 0 1) 
(1 1 1 0 1 0 0 0 0 1)

Program 3 output:
[1]> 
MIRROR?
[2]> 
T 
T

Program 4 output:
[1]> 
SHORTEST-PATH
[2]> 
BFS
[3]> 
NEW-PATHS
[4]> 
(A C D) 
(A C D)

Program 5 output:
[1]> 
BIN-SEARCH
[2]> 
FINDER
[3]> 
3 
3

Program 6 output:
[1]> 
SECOND-WORD
[2]> 
"follows" 
"follows"

Program 7 output:
[1]> 
TOKENS
[2]> 
CONSTITUENT
[3]> 
PARSE-DATE
[4]> 
MONTH-NAMES
[5]> 
PARSE-MONTH
[6]> 
("ab" "cde" "f") 
("ab" "cde" "f")
[7]> 
("ab12" "3cde.f" "gh") 
("ab12" "3cde.f" "gh")
[8]> 
(16 8 1980) 
(16 8 1980)

Program 8 output:
[1]> 
MY-READ-INTEGER
[2]> 
578 
578

Program 9 output:
[1]> 
NODE
[2]> 
BST-INSERT
[3]> 
BST-FIND
[4]> 
BST-MIN
[5]> 
BST-MAX
[6]> 
BST-TRAVERSE
[7]> 
BST-REMOVE
[8]> 
PERCOLATE
[9]> 
BST-REMOVE-MIN
[10]> 
BST-REMOVE-MAX
[11]> 
NIL
[12]> 
NIL
[13]> 
NIL 
NI

In [4]:

# save Lisp outputs to a file
with open("lisp_results.txt", "w", encoding="utf-8") as f:
    for i, result in enumerate(results, 1):
        f.write(f"Program {i} output:\n")
        f.write(result + "\n")
        f.write("-" * 40 + "\n")


In [5]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    api_key="", # insert HuggingFace API key
)

# run the LLM on each Lisp program and store the translated C program
c_programs = []
for i, program in enumerate(programs):
  completion = client.chat.completions.create(
      model="Qwen/Qwen2.5-Coder-32B-Instruct",
      messages = [
        {"role": "system", "content": """You are an agent tasked with translating Lisp code to idiomatic C code.

  Here is an example of what you will output, make sure to follow the format exactly:

  ;Lisp Input:
  (defun add (a b)
    (+ a b))
  (print (add 1 2))
  (print (add 5 4))
  (print (add 6 0))

  //C Output:
  int add(int a, int b){
    return a + b;
  }
  int main(){
    printf(add(1, 2));
    printf(add(5, 4));
    printf(add(6, 0));
  }

  Keep in mind:
      - The function calls in the must be valid and syntactically correct so that the user can directly execute them. This is very important, do NOT use pseudo-code or undefined functions.
      - All functions must be properly defined in the C output
      - The C output must be functionally equivalent to the Lisp input such that when both programs are executed they print EXACTLY the same output
      - Do not output anything else

  Final Reminders:
      - The C code MUST be immediately executable in C
      - Do NOT output any other text or explanations besides the code, even if it's important to the user"""
  },
        {"role": "user", "content": programs[i]+"\n\n//C Output:"}
      ],
  )

  c_programs.append(completion.choices[0].message.content)
  print(c_programs[i])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


```c
#include <stdio.h>
#include <stdlib.h>

typedef struct Node {
    int value;
    struct Node* next;
} Node;

Node* createNode(int value) {
    Node* newNode = (Node*)malloc(sizeof(Node));
    newNode->value = value;
    newNode->next = NULL;
    return newNode;
}

void appendNode(Node** head, int value) {
    Node* newNode = createNode(value);
    if (*head == NULL) {
        *head = newNode;
        return;
    }
    Node* temp = *head;
    while (temp->next != NULL) {
        temp = temp->next;
    }
    temp->next = newNode;
}

Node* compress(Node* x) {
    if (x == NULL) return NULL;
    return compr(x->value, 1, x->next);
}

Node* compr(int elt, int n, Node* lst) {
    if (lst == NULL) {
        if (n > 1) {
            Node* resultHead = createNode(n);
            resultHead->next = createNode(elt);
            return resultHead;
        } else {
            return createNode(elt);
        }
    }
    int next = lst->value;
    if (next == elt) {
        return compr(elt, n 

HfHubHTTPError: 504 Server Error: Gateway Time-out for url: https://router.huggingface.co/nscale/v1/chat/completions

In [6]:
# remove backticks from LLM output
c_programs = [
    prog.replace("```", "")
    for prog in c_programs
]

# remove 'C' from LLM output
for i in range(len(c_programs)):
    lines = c_programs[i].splitlines()
    lines = [line for line in lines if line.strip() != 'c']
    c_programs[i] = "\n".join(lines).strip()

# print preprocessed C programs
for i in c_programs:
  print(i)
  print("\n")

#include <stdio.h>
#include <stdlib.h>

typedef struct Node {
    int value;
    struct Node* next;
} Node;

Node* createNode(int value) {
    Node* newNode = (Node*)malloc(sizeof(Node));
    newNode->value = value;
    newNode->next = NULL;
    return newNode;
}

void appendNode(Node** head, int value) {
    Node* newNode = createNode(value);
    if (*head == NULL) {
        *head = newNode;
        return;
    }
    Node* temp = *head;
    while (temp->next != NULL) {
        temp = temp->next;
    }
    temp->next = newNode;
}

Node* compress(Node* x) {
    if (x == NULL) return NULL;
    return compr(x->value, 1, x->next);
}

Node* compr(int elt, int n, Node* lst) {
    if (lst == NULL) {
        if (n > 1) {
            Node* resultHead = createNode(n);
            resultHead->next = createNode(elt);
            return resultHead;
        } else {
            return createNode(elt);
        }
    }
    int next = lst->value;
    if (next == elt) {
        return compr(elt, n + 1, 

In [7]:
# save C programs to a file
with open("c_programs.txt", "w", encoding="utf-8") as f:
    for i, prog in enumerate(c_programs, 1):
        f.write(f"Program {i}:\n")
        f.write(prog + "\n")
        f.write("-" * 40 + "\n")

In [8]:
import subprocess
import tempfile
import os
c_results = []

# compile and run each C program and store the outputs
for i, program in enumerate(c_programs, 1):
    # create a temporary C source file
    with tempfile.NamedTemporaryFile(suffix=".c", delete=False) as src_file:
        src_file.write(program.encode())
        src_path = src_file.name

    # create a temporary file for the compiled executable
    exe_path = src_path[:-2]

    # compile the C program
    compile_proc = subprocess.run(
        ["gcc", src_path, "-o", exe_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    if compile_proc.returncode != 0:
        # compilation failed
        c_results.append(f"Compilation Error:\n{compile_proc.stderr.decode()}")
        # clean up source file
        os.remove(src_path)
        continue

    # run the compiled executable
    run_proc = subprocess.run(
        [exe_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    output = run_proc.stdout.decode().strip()
    error  = run_proc.stderr.decode().strip()

    # append result
    c_results.append(output if output else f"[ERROR] {error}" if error else "")

    # clean up temporary files
    os.remove(src_path)
    os.remove(exe_path)

In [ ]:
# print out the C program outputs
for i, result in enumerate(c_results, 1):
    print(f"{i}: {result}")

1: Compilation Error:
/tmp/tmpdx1iqxv0.c: In function ‘print_list’:
/tmp/tmpdx1iqxv0.c:19:17: error: expected expression before ‘typeof’
   19 |             if (typeof(lst->value) == typeof(int*)) {
      |                 ^~~~~~
/tmp/tmpdx1iqxv0.c: In function ‘compress’:
/tmp/tmpdx1iqxv0.c:66:22: error: expected expression before ‘typeof’
   66 |     if (x != NULL && typeof(x->value) == typeof(int*)) {
      |                      ^~~~~~
/tmp/tmpdx1iqxv0.c: In function ‘main’:
/tmp/tmpdx1iqxv0.c:73:165: error: expected ‘)’ before ‘;’ token
   73 |     List* list = cons((void*)1, cons((void*)1, cons((void*)1, cons((void*)0, cons((void*)1, cons((void*)0, cons((void*)0, cons((void*)0, cons((void*)1, NULL))))))));
      |                      ~                                                                                                                                              ^
      |                                                                                                 

In [9]:
# save the C outputs to a file
with open("c_results.txt", "w", encoding="utf-8") as f:
    for i, result in enumerate(c_results, 1):
        f.write(f"Program {i} output:\n")
        f.write(result + "\n")
        f.write("-" * 40 + "\n")